In [1]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from PIL import Image
import os
# from tqdm.notebook import tqdm
from tqdm import tqdm
torch.backends.cuda.matmul.allow_tf32 = False
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

In [2]:
class CFG():
    def __init__(self) -> None:
        self.num_workers = 0
        self.batch_size = 32
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.root_dir = 'D:/data/cow_teat_segmentation'
        self.model_save_dir = 'D:/data/cow_teat_segmentation/model/deeplabV3+/efficientb0'
        self.learning_rate = 1e-5
        self.weight_decay = 1e-5
        
        self.encoder_name="resnet34",           # choose encoder of model, e.g. resnet34, resnet50, resnet101, resnet152
        self.encoder_weights="imagenet",        # use `imagenet` pretrained weights for encoder initialization
        self.if_load_model = True
        self.model_path = self.model_save_dir+'/best_model.pth'
        
        self.val_transform = A.Compose([
            A.Resize(224, 224),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
        
        self.transform = A.Compose([
            A.Resize(224, 224),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
            A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(), 
                A.MotionBlur(),
                ], p=0.4),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
        
cfg = CFG()

transform = cfg.transform


In [3]:
if not os.path.exists(cfg.model_save_dir):
    os.makedirs(cfg.model_save_dir+'/')

In [4]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'


In [5]:
root_train= cfg.root_dir + "/Training"
root_valid= cfg.root_dir + "/Valid"

In [6]:
class SegmentationDataset():
  def __init__(self, root_dir, transform=None):
    self.root_dir = root_dir
    self.transform = transform
    self.image_list = os.listdir(os.path.join(self.root_dir,"Images"))
    self.mask_list = os.listdir(os.path.join(self.root_dir,"Masks"))
  
  def __len__(self):
    return len(self.image_list)

  def __getitem__(self, idx):
    img_path= os.path.join(self.root_dir,"Images",self.image_list[idx])
    mask_path=os.path.join(self.root_dir,"Masks" ,self.mask_list[idx])

    # image = np.asarray(Image.open(img_path).convert ("RGB"))
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # mask = np.asarray(Image.open(mask_path).convert('L'))
    mask = cv2.imread(mask_path, 0)

    if self. transform:
      transformed = self.transform(image=image, mask=mask)
      image = transformed['image']
      mask = transformed['mask']

    return image, mask


# Define the dataset and data loader

transform = cfg.transform
val_transform = cfg.val_transform


In [7]:
train_loader_params = {
    'batch_size': cfg.batch_size,
    'shuffle': True,
    'num_workers': cfg.num_workers,
    'pin_memory': True,
    'drop_last': True,
}

train_dataset = SegmentationDataset(root_train, transform=transform)
train_loader = DataLoader(train_dataset, **train_loader_params)

In [8]:
valid_loader_params = {
    'batch_size': cfg.batch_size,
    'shuffle': False,
    'num_workers': cfg.num_workers,
    'pin_memory': True,
}
valid_dataset = SegmentationDataset(root_valid, transform=val_transform)
valid_loader = DataLoader(valid_dataset, **valid_loader_params)

In [9]:
len(train_dataset)

31920

In [10]:
train_dataset[0][1].shape

torch.Size([224, 224])

In [11]:
for images, masks in train_loader:
    for image, mask in zip(images, masks):
        print('Image dimensions:', image.shape)
        print('Mask dimensions:', mask.shape)
        break
    break

Image dimensions: torch.Size([3, 224, 224])
Mask dimensions: torch.Size([224, 224])


In [12]:
import segmentation_models_pytorch as smp
model = smp.DeepLabV3Plus(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pretrained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=65,                      # model output channels (number of classes in your dataset)
)
if cfg.if_load_model:
    model.load_state_dict(torch.load(cfg.model_path))
    print('load model from', cfg.model_path)
num_classes=65
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

load model from D:/data/cow_teat_segmentation/model/deeplabV3+/efficientb0/best_model.pth


In [13]:
lr = cfg.learning_rate


criterion = nn.CrossEntropyLoss()
weight_decay = cfg.weight_decay
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)


In [14]:
len(train_loader)

997

In [15]:
# validate(model, valid_loader, criterion, device)

# Predicted mask comparison with ground truth mask

In [16]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_iou = 0.0
    running_dice = 0.0
    iou = 0
    dice = 0
    loss = 0
    # tqdm._instances.clear()
    bar = tqdm(val_loader, desc=f"Val Progress -  iou: {iou:.3f}, dice: {dice:.3f},loss:{loss:.5f}", leave=False)
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(bar):
            bar.set_description(f"Val Progress -  iou: {iou:.3f}, dice: {dice:.3f},loss:{loss:.5f}")
            # print("\r val {:2}%".format((i+1)/len(val_loader)),end="")
            inputs = inputs.to(device)
            labels = labels.to(device).long()

            # One-hot encode the labels

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            # Compute IoU and dice scores
            predicted = torch.argmax(outputs, dim=1)
            # labels = labels.squeeze(1)
            union = torch.logical_or(predicted, labels).sum()
            intersection = torch.logical_and(predicted, labels).sum()
            if union == 0 :
                iou = 1
                dice = 1
            else:
                iou = intersection / union
                dice = iou*2/(iou+1)
            running_iou += iou
            running_dice += dice
    epoch_loss = running_loss / len(val_loader)
    epoch_iou = running_iou / len(val_loader)
    epoch_dice = running_dice / len(val_loader)
    
    print(f'Validation Loss: {epoch_loss:.4f} | IoU: {epoch_iou:.4f} | Dice: {epoch_dice:.4f}')
    # print(1)
    return epoch_dice


# Training the model for 3 epoch

In [17]:
def save_model(model, save_path):
    torch.save(model.state_dict(), save_path)

In [18]:
torch.cuda.empty_cache()

In [19]:

def train(model, train_loader, optimizer, criterion, device, num_epochs=3):
    best_dice = validate(model, valid_loader, criterion, device)
    best_dice = 1
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        running_dice = 0.0
        iou = 0
        dice = 0
        loss = 0
        tqdm._instances.clear()
        bar = tqdm(train_loader, desc=f"Training Progress - Epoch: {epoch+1}/{num_epochs}, iou: {iou:.3f}, dice: {dice:.3f}")
        # bar = tqdm(valid_loader, desc=f"Val Progress -  iou: {iou:.3f}, dice: {dice:.3f},loss:{loss:.5f}")
        for i, (inputs, labels) in enumerate(bar):
            bar.set_description(f"Training Progress - Epoch: {epoch+1}/{num_epochs}, iou: {iou:.3f}, dice: {dice:.3f},loss:{loss:.5f}")
            # bar.set_description(f"Val Progress -  iou: {iou:.3f}, dice: {dice:.3f},loss:{loss:.5f}")
            inputs = inputs.to(device)
            labels = labels.to(device).long()

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            predicted = torch.argmax(outputs, dim=1)
            union = torch.logical_or(predicted, labels).sum()
            intersection = torch.logical_and(predicted, labels).sum()
            if union == 0 :
                iou = 1
                dice = 1
            else:
                iou = intersection / union
                dice = iou*2/(iou+1)
            # dice = (2 * intersection / (predicted_onehot.sum(dim=(1,2)) + labels_onehot.sum(dim=(1,2)))).mean().item()

            running_iou += iou
            running_dice += dice
        
        epoch_loss = running_loss / len(train_loader)
        epoch_iou = running_iou / len(train_loader)
        epoch_dice = running_dice / len(train_loader)
        val_dice = validate(model, valid_loader, criterion, device)
        if val_dice < best_dice:
            model_path= cfg.model_save_dir+'/best_model.pth'
            save_model(model, model_path)
            best_dice = val_dice
        print(f'Epoch: {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f} | IoU: {epoch_iou:.4f} | Dice: {epoch_dice:.4f}')
        print()

In [20]:
train(model, train_loader, optimizer, criterion, device, num_epochs=10)

Validation Loss: 0.0134 | IoU: 0.4273 | Dice: 0.5860


Training Progress - Epoch: 1/10, iou: 0.804, dice: 0.891,loss:0.00226:   3%|▎         | 32/997 [00:08<04:07,  3.89it/s]


KeyboardInterrupt: 

In [ ]:
train(model, train_loader, optimizer, criterion, device, num_epochs=10)

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0131 | IoU: 0.1772 | Dice: 0.2805


Training Progress - Epoch: 1/10, iou: 0.000, dice: 0.000:   0%|          | 0/997 [00:00<?, ?it/s]

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0123 | IoU: 0.3266 | Dice: 0.4765
Epoch: 1/10, Training Loss: 0.0106 | IoU: 0.4850 | Dice: 0.6516



...

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0133 | IoU: 0.4375 | Dice: 0.5954
Epoch: 9/10, Training Loss: 0.0031 | IoU: 0.7346 | Dice: 0.8467



Training Progress - Epoch: 10/10, iou: 0.000, dice: 0.000:   0%|          | 0/997 [00:00<?, ?it/s]

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0130 | IoU: 0.4401 | Dice: 0.5981
Epoch: 10/10, Training Loss: 0.0029 | IoU: 0.7441 | Dice: 0.8530



In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr/10, weight_decay=weight_decay)
train(model, train_loader, optimizer, criterion, device, num_epochs=20)

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0130 | IoU: 0.4401 | Dice: 0.5981


Training Progress - Epoch: 1/20, iou: 0.000, dice: 0.000:   0%|          | 0/997 [00:00<?, ?it/s]

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0132 | IoU: 0.4375 | Dice: 0.5960
Epoch: 1/20, Training Loss: 0.0028 | IoU: 0.7531 | Dice: 0.8590



...

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0135 | IoU: 0.4326 | Dice: 0.5915
Epoch: 19/20, Training Loss: 0.0025 | IoU: 0.7703 | Dice: 0.8701



Training Progress - Epoch: 20/20, iou: 0.000, dice: 0.000:   0%|          | 0/997 [00:00<?, ?it/s]

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0135 | IoU: 0.4274 | Dice: 0.5863
Epoch: 20/20, Training Loss: 0.0025 | IoU: 0.7706 | Dice: 0.8702



In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr/100, weight_decay=weight_decay)
train(model, train_loader, optimizer, criterion, device, num_epochs=10)

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0135 | IoU: 0.4274 | Dice: 0.5863


Training Progress - Epoch: 1/10, iou: 0.000, dice: 0.000:   0%|          | 0/997 [00:00<?, ?it/s]

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0134 | IoU: 0.4273 | Dice: 0.5860
Epoch: 1/10, Training Loss: 0.0025 | IoU: 0.7719 | Dice: 0.8711



...

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0134 | IoU: 0.4273 | Dice: 0.5860
Epoch: 9/10, Training Loss: 0.0025 | IoU: 0.7719 | Dice: 0.8711



Training Progress - Epoch: 10/10, iou: 0.000, dice: 0.000:   0%|          | 0/997 [00:00<?, ?it/s]

Val Progress -  iou: 0.000, dice: 0.000,loss:0.00000:   0%|          | 0/91 [00:00<?, ?it/s]

Validation Loss: 0.0136 | IoU: 0.4276 | Dice: 0.5861
Epoch: 10/10, Training Loss: 0.0025 | IoU: 0.7728 | Dice: 0.8717



# test model

# Saving the model and also loading the same model

In [ ]:
# import torch

# # Load the saved model
# model = torch.load('/kaggle/working/model.pth')


In [ ]:
# # Define the model architecture
# model = UNet(n_classes=65)

# # Load the state dictionary
# state_dict = torch.load('/kaggle/working/model.pth',map_location=torch.device('cuda'))
# model.load_state_dict(state_dict)
# model.eval()


# # Load the state dictionary to the model instance
